为了实现良好解耦的多模态隐空间，我们引入了互信息（MI）理论来分离共享和私有表征 1。该模块的损失函数定义如公式 (10)：$$L_{CMI} = \sum_{i \in {r, a}} I(z_{p_i}, z_{s_i}) + I(z_{p_r}, z_{p_a})$$此处，$I(z_{p_i}, z_{s_i})$ 测量单一模态 $i$ 内私有和共享隐嵌入之间的 MI，$I(z_{p_r}, z_{p_a})$ 测量跨模态的私有嵌入之间的 MI 1。目标是最小化模态内共享与私有嵌入之间、以及跨模态私有嵌入之间的 MI 1。两个变量 $x$ 和 $y$ 之间的 MI 定义如公式 (11)：$$I(x; y) = E_{p(x, y)} [log \frac{p(x, y)}{p(x)p(y)}]$$由于互信息通常是难以计算的，我们使用一个称为对比对数率上界（CLUB）的上界 1。$I_{CLUB}(x, y)$ 的近似描述如公式 (12)：$$I_{CLUB}(x, y) = E_{p(x, y)} [log p(y|x)] - E_{p(x)} E_{p(y)} [log p(y|x)]$$为了验证 $I_{CLUB}(x, y)$ 是 $I(x; y)$ 的上界，我们计算它们之间的差异 $\Delta = I_{CLUB} - I(x; y)$ 如公式 (13)：$$\Delta = E_{p(y)} [log p(y)] - E_{p(x)} [log p(y|x)]$$使用边缘分布 $log p(y) = log \int p(y|x)p(x)dx = log(E_{p(x)} [p(y|x)])$ 和詹森不等式，我们推导出：$$log p(y) = log E_{p(x)} [p(y|x)] \ge E_{p(x)} [log p(y|x)]$$这表明 $\Delta \ge 0$，确认 $I_{CLUB}(x, y)$ 确实是 $I(x; y)$ 的上界，其计算方式为：$$I_{CLUB} = \frac{1}{N} \sum_{i=1}^N log p(y_i|x_i) - \frac{1}{N^2} \sum_{i=1}^N \sum_{j=1}^N log p(y_j|x_i)$$简化为：$$I_{CLUB} = \frac{1}{N^2} \sum_{i=1}^N \sum_{j=1}^N [log p(y_i|x_i) - log p(y_j|x_i)]$$我们采用非线性神经网络来估计 $log p(y|x)$，对数似然计算为：$$\sum_{i=1}^N log p(y_i|x_i; \phi)$$在每个训练周期中，分布估计器的参数首先被更新以最大化公式 (17)，从而提高条件分布估计的准确性 1。随后，scTFBridge 的其他参数被更新以最小化 $I_{CLUB}$，从而降低变量之间的互信息。